**Franco Ortiz** :Data scientist

UFO-reports-analysis
# Problem Statement
### Questions to answer
- What time is more frequent to see them?
- How long did the experience take?
- What shapes are most common and where?
- Top words that described that moment (wordcloud)
- heat map with quantities and cities for number of sightings (geopandas and foliumn)

# Data importing
#### modules

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from dataprep.clean import clean_date
import re

# set seed for reproducibility
np.random.seed(0)

In [2]:
df = pd.read_csv("data/ufo.csv", keep_default_na=True)
df.head(20)

,Date / Time,City,State,Country,Shape,Duration,Summary,Posted,Images
0,00:00,Dieppe (Moncton) (Canada),NB,Canada,Oval,NaN,((HOAX??)) we wher looking .. in the sky in di...,9/24/12,NaN
1,4/ 23:22,Atlantic City,NJ,USA,Disk,30 minutes,Observed a very bright white saucer (had two l...,5/6/17,NaN
2,03142020 23:04,Norwich,VT,USA,Circle,97 seconds,Strange sighting in the sky,5/15/20,NaN
3,1970s 23:00,Grande Summit,PA,USA,Light,5 minutes,4 lights speeding very fast across sky silently,8/16/21,NaN
4,23:00,Desolation Wilderness,CA,USA,Triangle,2 min.,triangle of lights moving slowly through the n...,9/15/05,NaN
5,23:00,Playa del Rey,CA,USA,Fireball,3-5 seconds,"Neon green, long-tailed fireball meteor-lookin...",7/8/04,NaN
6,23:00,Milltown (Canada),NB,Canada,Circle,NaN,"I wonder where they are today, and if they are...",6/4/04,NaN
7,23:00,Manning,SC,USA,Sphere,2 minutes,The object was metallic in color. Spherical in...,6/25/20,NaN
8,23:00,New Alexandria,PA,USA,Other,15 minutes,Translucent UFO.,5/9/19,NaN
9,23:00,"Northern California Coast (in, highway)",CA,USA,Cigar,3-5 min.,We were travelling along the coast of Californ...,10/2/99,NaN


- ***As we see index 1,2,3,4 must be put in an standard format***

In [3]:
df.columns

Index(['Date / Time', 'City', 'State', 'Country', 'Shape', 'Duration',
       'Summary', 'Posted', 'Images'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138298 entries, 0 to 138297
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Date / Time  138197 non-null  object
 1   City         137909 non-null  object
 2   State        128886 non-null  object
 3   Country      138075 non-null  object
 4   Shape        132096 non-null  object
 5   Duration     131537 non-null  object
 6   Summary      138227 non-null  object
 7   Posted       138296 non-null  object
 8   Images       524 non-null     object
dtypes: object(9)
memory usage: 9.5+ MB


# Data Cleaning

In [5]:
df['Date / Time']

0                  00:00
1               4/ 23:22
2         03142020 23:04
3            1970s 23:00
4                  23:00
               ...      
138293      5/1/22 06:00
138294      5/1/22 04:41
138295      5/1/22 04:25
138296      5/1/22 04:00
138297      5/1/22 02:37
Name: Date / Time, Length: 138298, dtype: object

#### splitting datetime column

In [ ]:
# Create a copy of the dataframe
df_clean = df.copy()

# Try to convert the 'Date / Time' column to datetime format
df_clean['Date / Time'] = pd.to_datetime(df_clean['Date / Time'], errors='coerce')

# Check for invalid dates and handle them
df_clean = df_clean[pd.notnull(df_clean['Date / Time'])]

# Extract the date and time separately into separate columns
df_clean['Date'] = df_clean['Date / Time'].dt.date
df_clean['Time'] = df_clean['Date / Time'].dt.time

# Drop the original 'Date / Time' column
df_clean.drop(columns=['Date / Time'], inplace=True)

#### column values to lowercase

In [ ]:
for column in df.columns:
    df[column] = df[column].str.lower()
df.head()

#### cleaning Summary column

In [ ]:
# return only words if them are str, if not '', using regular expression
# (r'\b\w+\b', x) words and digits
df['words'] = df['Summary'].apply(lambda x: re.findall(r'[^\d\W]+', x) if isinstance(x, str) else [])
df['words']

#### cleaning Duration column

In [ ]:
df['Duration'].head(10)

In [ ]:
df['Duration'] = df['Duration'].astype(str)
df['Duration'] = df['Duration'].str.extract(r'(\d+)').astype(float)
df['Duration'] = df['Duration'].fillna(0)
df['Duration'] = df['Duration'].apply(lambda x: x * 60 if x < 100 else x)
df['Duration'] = df['Duration'].astype(int)
df['Duration'].head(10)

# EDA

### what shapes are most common and where 

In [ ]:
sns.countplot(y ='Shape', 
              data = df,
              color='limegreen',
              order = df['Shape'].value_counts().index)
plt.show()

In [ ]:
# Top 15 places
shapes_where = df.groupby(['City'])['Shape'].value_counts().sort_values(ascending=False).head(15)
shapes_where

- ***Seems like most common shape is 'light-shaped', followe by circle and triangle. The top 3 places they are seen is in Seattle, Phoenix, Las Vegas***

### Most used words in description
Text Mining with NLTK, and frecuency distribution

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import PIL.Image

In [ ]:
# for WordCloud.generate() method should be a single string
words_string = ' '.join(df['words'].str.join(' '))

In [ ]:
wc = WordCloud(background_color='white', max_font_size=40)
wc.generate(words_string)

In [ ]:
personalized_stopwords = ["nuforc", "one", "two", "three", "see", "saw", "seen", "looked", "light", "white light", "red light"]

In [ ]:
mask = np.array(PIL.Image.open("img/ufo.jpg"))

wc = WordCloud(stopwords = STOPWORDS.union(set(personalized_stopwords)),
               mask = mask,
               background_color = "white",
               contour_color = "black",
               contour_width = 0.9,
               min_font_size=5).generate(words_string)

plt.imshow(wc.to_array(), interpolation='bilinear')
plt.axis("off")
plt.show()

- ***As we can see most frequent words are :*** 
        ***Sky, Object, bright light, star, hovering, sound, ufo, night sky***

#### What time is most frequent to see them?

In [ ]:
top_15_times = df_clean['Time'].value_counts().sort_values(ascending=False).head(15)

In [ ]:
top_15_times.plot(kind='bar',figsize=(7, 3),color='limegreen')

plt.xlabel('Time')
plt.ylabel('Counts')
plt.title('Top 15 Time with highest counts')
plt.xticks(rotation=50)

plt.show()

- ***most frequent hours are from 21 to 23 hs***

#### How long did the experience take?

In [ ]:
df.head()

In [ ]:
#Count the occurrences of each value in the Duration column
value_counts = sorted_duration['Duration'].value_counts()

#Create the violin plot
value_counts.plot(kind='bar',figsize=(7, 3),
                 color='limegreen')
plt.xlim(0, 15)

#Add a title and axis labels
plt.title("Distribution of UFO Sighting Durations")
plt.xlabel("Duration (in minutes)")
plt.ylabel("Count")
plt.xticks(rotation=50)

#Show the plot
plt.show()

In [ ]:
print('mean duration: ',value_counts.mean())
print('minimun duration: ',value_counts.min())

- ***duration tends to be:  300 seconds ->  around 5 min***
- ***mean duration:         794 seconds ->13 min***
- ***minimun duration :     1 second***

# Modelling

#### Heat map with quantities and cities for number of sightings

In [ ]:
df_clean.head()

In [ ]:
df_clean['City']

In [ ]:
import geopandas as gpd
import folium

# Convert to DataFrame
df_pd = pd.DataFrame(df_clean)

# Convert the dataframe into a Geopandas dataframe
gdf = gpd.GeoDataFrame(df_pd, geometry=[Point(0, 0) for i in range(5)]) #add latitude and long

''' In this example, all the sightings are represented by a 
single point at (0, 0) for simplicity. To accurately plot the locations, you'll need to add latitude and longitude columns to the dataframe.'''
# Plot the data on a Folium map
m = folium.Map(location=[45.5236, -122.6750], zoom_start=13)
folium.GeoJson(gdf).add_to(m)
m.save('map.html')

# Communication